# Plotting of  Aluminosilicate Phase Diagram from Berman(1988) database of pure solid phases (standard state properties)
## Data are adjusted datapoints from Holdaway1993
## Concepts Demonstrated
* Loading a thermodynamic database
* Imposing experimental priors

## Import libraries ...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook
import matplotlib as mpl
import pandas as pd
pd.set_option('display.max_columns', None)
from scipy import stats

import os
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '/Users/msghiorso/ENKI/PhaseObjC')

import thermoengine
from thermoengine import phases
from thermoengine import model

import importlib


## Define reusable functions for analysis and visualization
* improve code readability
* reduce coding errors
* speed up debugging

In [ ]:
def get_nice_colors(Ncolor,whitefrac=0.0):
    cmap = plt.get_cmap(name='viridis')
    ind_a = np.int_(np.round(np.linspace(0,255,Ncolor)))
    col_a = np.array([cmap.colors[ind] for ind in ind_a])
    col_a = (1-whitefrac)*col_a+whitefrac*np.ones(col_a.shape)
    return col_a



In [ ]:
def plot_phase_diagram(rxns, rxn_eqns, fig=None, Tlims=[200, 1100], 
                       show_legend=True):
    if fig is None:
        fig = plt.figure()
        
    plt.figure(fig.number)
    colors = get_nice_colors(3)
    
    Tlims = np.array(Tlims)

    rxn_bnds = []
    T_bnds = []
    P_bnds = []

    for irxn, irxn_eqn, col in zip(rxns, rxn_eqns, colors):
        iTbnds, iPbnds = irxn.trace_boundary(Tlims=Tlims+273.15, Nsamp=100)
        T_bnds.append(iTbnds)
        P_bnds.append(iPbnds)
        istable = irxn.stability(iTbnds, iPbnds, rxns)
    
        # irxn_bnd, = plt.plot(iTbnds-273.15, iPbnds/1e3, lw=2, color=col)
        irxn_bnd, = plt.plot(iTbnds[istable]-273.15, iPbnds[istable]/1e3, '-',
                             lw=2, color=col)
        irxn_bnd_meta, = plt.plot(iTbnds[~istable]-273.15, iPbnds[~istable]/1e3, '--',
                             lw=2, color=col)
    
        rxn_bnds.append(irxn_bnd)
   
    

    plt.xlabel('Temp.  [C]')
    plt.ylabel('Press.  [kbar]')

    # plt.plot(data_adj['data']['T'][1:], data_adj['data']['P'][1:], 'k+')

    if show_legend:
        plt.legend(rxn_bnds, rxn_eqns)

    # plt.ylim(-0., 18)
    plt.xlim(Tlims)
    return fig

def plot_rxn_data(df, fig=None):
    if fig is None:
        fig = plt.figure()
        
    plt.figure(fig.number)
    colors = get_nice_colors(3)

    for irxn, irxn_eqn, col in zip(rxns, rxn_eqns, colors):   
        imask_fwd = (df['rxn_studied']==irxn_eqn) & (df['rxn_dir']==1)
        imask_rev = (df['rxn_studied']==irxn_eqn) & (df['rxn_dir']==-1)
        
        plt.plot(df['T'][imask_fwd], df['P'][imask_fwd], 'v', 
                 color=col, ms=4, mfc='None')
        plt.plot(df['T'][imask_rev], df['P'][imask_rev], '^', 
                 color=col, ms=4, mfc='None')
    
    return fig


In [ ]:
modelDB = model.Database()

In [ ]:
rxn_coef = [-1, +1]
rxn_endmem = [0, 0]

# Create reactions
rxns = []
rxns.append(modelDB.get_rxn(['And','Ky'], rxn_endmem, rxn_coef))
rxns.append(modelDB.get_rxn(['And','Sil'], rxn_endmem, rxn_coef))
rxns.append(modelDB.get_rxn(['Sil','Ky'], rxn_endmem, rxn_coef))

# Create labels for legend
rxn_eqns = []
rxn_eqns.append('And = Ky')
rxn_eqns.append('And = Sil')
rxn_eqns.append('Sil = Ky')


In [ ]:
# Read data
data_adj = pd.read_excel(
    'phase-rxn-data/Al2SiO5-calib-data-Holdaway1993.xlsx',
    sheet_name=None)


df = data_adj['data'][1:].reset_index(drop=True)
rxn_dir = pd.Series(np.zeros(df.shape[0]))

for irxn, irxn_eqn in zip(rxns, rxn_eqns):
    imask = df['rxn_studied']==irxn_eqn
    iresults = pd.DataFrame([pd.Series(df[imask]['results'].str.startswith(iphs),name=iphs) 
                         for iphs in irxn.phase_symbols]).T
    
    irxn_dir = np.dot(irxn.rxn_coefs, 
                      np.array([df[imask]['results'].str.startswith(iphs)
                                for iphs in irxn.phase_symbols]))
    rxn_dir[imask] = irxn_dir
    
df['rxn_dir'] = rxn_dir

In [ ]:
fig = plot_phase_diagram(rxns, rxn_eqns, Tlims=[200, 1100], 
                         show_legend=True)
fig = plot_rxn_data(df, fig=fig)